In [5]:
%load_ext tensorboard

In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding, GRU, Activation, Dropout, Dense, Input, Bidirectional, Layer
from tensorflow.keras.models import Model, Sequential
from keras.backend import clear_session
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.activations import tanh
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from tensorflow import keras

import keras.backend as kb
import tensorflow
from tensorboard.plugins.hparams import api as hp

In [6]:
dataset = pickle.load(open("dataset/dataset_IMDB.pickle", "rb"))

In [7]:
Y = np.array(list(map(lambda x: 1 if x=="positive" else 0, dataset["sentiment"])))

In [8]:
x_train,x_test,y_train,y_test = train_test_split(dataset["processed_review"],Y, test_size=0.1, shuffle=True)

In [10]:
print("numero di positivi nel train: ", list(y_train).count(1))
print("numero di negativi nel train: ", list(y_train).count(0))

print("numero di positivi nel test: ", list(y_test).count(1))
print("numero di negativi nel test: ", list(y_test).count(0))

numero di positivi nel train:  22479
numero di negativi nel train:  22521
numero di positivi nel test:  2521
numero di negativi nel test:  2479


In [9]:
def creazione_modello_GloVe(filename):
    f = open(filename, encoding="utf8")
    embeding_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeding_index[word] = coefs
    f.close()
    return embeding_index

In [11]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [12]:
tokenizer = Tokenizer(num_words=142092)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [13]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banana"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=200, weights=[embedding_matrix])

In [14]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=kb.squeeze(kb.tanh(kb.dot(x,self.W)+self.b),axis=-1)
        at=kb.softmax(et)
        at=kb.expand_dims(at,axis=-1)
        output=x*at
        return kb.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [12]:
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(["adam", "sgd", "rmsprop"]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1,0.2, 0.3, 0.5]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64]))

log_dir = 'logs/IMDB_BiGRUAtt_HP'
METRIC_ACCURACY = 'accuracy'

with tensorflow.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Accuracy")],
    )

In [13]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(45000,) (45000,)
(5000,) (5000,)


In [15]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=200)

In [15]:
print(x_train_index.shape)

(45000, 200)


In [16]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)
y_test_categorical = keras.utils.to_categorical(y_test, 2)

In [17]:
print(y_train_categorical.shape)
print(y_test_categorical.shape)

(45000, 2)
(5000, 2)


In [17]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=200)

In [19]:
def train_test_model(hparams):
    model = Sequential([
        embedding_layer,
        Bidirectional(GRU(units=hparams[HP_NUM_UNITS], return_sequences=True, dropout=hparams[HP_DROPOUT])),
        Dense(hparams[HP_NUM_UNITS]),
        Dense(hparams[HP_NUM_UNITS]),
        attention(),
        Dense(2, activation="sigmoid"),
    ])
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(x_train_index[:35000], y_train_categorical[:35000], epochs=10, batch_size=128, verbose=1, validation_data=(x_train_index[35000:], y_train_categorical[35000:]), callbacks=[tensorflow.keras.callbacks.TensorBoard(log_dir), hp.KerasCallback(log_dir, hparams)])
    _, accuracy = model.evaluate(x_test_index, y_test_categorical)
    return accuracy

In [20]:
def run(run_dir, hparams):
    with tensorflow.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_test_model(hparams)
        tensorflow.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [21]:
n_esecuzione = 0
for num_units in HP_NUM_UNITS.domain.values:
        for dropout in HP_DROPOUT.domain.values:
            for opt in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_DROPOUT: dropout,
                    HP_NUM_UNITS: num_units,
                    HP_OPTIMIZER: opt,
                    }
                print("esecuzione: ", n_esecuzione)
                print({h.name: hparams[h] for h in hparams})
                run(log_dir + "/esecuzione_{}".format(n_esecuzione), hparams)
                n_esecuzione += 1

esecuzione:  25
{'dropout': 0.1, 'num_units': 64, 'optimizer': 'adam'}
Epoch 1/10
274/274 [==============================] - 177s 615ms/step - loss: 0.4144 - accuracy: 0.8105 - val_loss: 0.3183 - val_accuracy: 0.8629
Epoch 2/10
274/274 [==============================] - 170s 620ms/step - loss: 0.2565 - accuracy: 0.8932 - val_loss: 0.2839 - val_accuracy: 0.8814
Epoch 3/10
274/274 [==============================] - 173s 633ms/step - loss: 0.1901 - accuracy: 0.9251 - val_loss: 0.2980 - val_accuracy: 0.8782
Epoch 4/10
274/274 [==============================] - 177s 647ms/step - loss: 0.1315 - accuracy: 0.9503 - val_loss: 0.3321 - val_accuracy: 0.8821
Epoch 5/10
274/274 [==============================] - 181s 659ms/step - loss: 0.0722 - accuracy: 0.9751 - val_loss: 0.3860 - val_accuracy: 0.8840
Epoch 6/10
274/274 [==============================] - 183s 668ms/step - loss: 0.0380 - accuracy: 0.9868 - val_loss: 0.5347 - val_accuracy: 0.8776
Epoch 7/10
274/274 [==============================] -

Dopo aver constatato che la migliore configurazione è: 
Num_units: 16, dropout=0.1, opt=sgd con accuracy del 88% si è deciso di provare a modificare il learning rate

In [19]:
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.005, 0.010, 0.015, 0.020, 0.050, 0.1]))

log_dir = 'logs/IMDB_BiGRUAtt_LR'
METRIC_ACCURACY = 'accuracy'

with tensorflow.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_LEARNING_RATE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Accuracy")],
    )

In [22]:
def train_test_model(hparams):
    model = Sequential([
        embedding_layer,
        Bidirectional(GRU(units=16, return_sequences=True, dropout=0.1)),
        Dense(16),
        Dense(16),
        attention(),
        Dense(2, activation="sigmoid"),
    ])
    opt = SGD(learning_rate=hparams[HP_LEARNING_RATE])
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(x_train_index[:35000], y_train_categorical[:35000], epochs=10, batch_size=128, verbose=1, validation_data=(x_train_index[35000:], y_train_categorical[35000:]), callbacks=[tensorflow.keras.callbacks.TensorBoard(log_dir), hp.KerasCallback(log_dir, hparams)])
    y_pred = model.predict(x_test_index)
    print(classification_report(y_test, np.argmax(y_pred, axis=1).astype("float32")))
    _, accuracy = model.evaluate(x_test_index, y_test_categorical)
    return accuracy

In [20]:
def run(run_dir, hparams):
    with tensorflow.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_test_model(hparams)
        tensorflow.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [23]:
n_esecuzione = 0
for lr in HP_LEARNING_RATE.domain.values:
    hparams = {
        HP_LEARNING_RATE: lr,
    }
    print("esecuzione: ", n_esecuzione)
    print({h.name: hparams[h] for h in hparams})
    run(log_dir + "/esecuzione_{}".format(n_esecuzione), hparams)
    n_esecuzione += 1 

esecuzione:  0
{'learning_rate': 0.005}
Epoch 1/10
274/274 [==============================] - 45s 154ms/step - loss: 0.6943 - accuracy: 0.4878 - val_loss: 0.6935 - val_accuracy: 0.4906
Epoch 2/10
274/274 [==============================] - 42s 152ms/step - loss: 0.6933 - accuracy: 0.5054 - val_loss: 0.6930 - val_accuracy: 0.5208
Epoch 3/10
274/274 [==============================] - 40s 147ms/step - loss: 0.6928 - accuracy: 0.5283 - val_loss: 0.6926 - val_accuracy: 0.5356
Epoch 4/10
274/274 [==============================] - 41s 151ms/step - loss: 0.6924 - accuracy: 0.5433 - val_loss: 0.6922 - val_accuracy: 0.5499
Epoch 5/10
274/274 [==============================] - 40s 148ms/step - loss: 0.6920 - accuracy: 0.5488 - val_loss: 0.6918 - val_accuracy: 0.5630
Epoch 6/10
274/274 [==============================] - 40s 146ms/step - loss: 0.6916 - accuracy: 0.5681 - val_loss: 0.6914 - val_accuracy: 0.5707
Epoch 7/10
274/274 [==============================] - 41s 151ms/step - loss: 0.6912 - accu